In [221]:
import pandas as pd

### функции для конверсий

In [222]:
def make_group(event: str):
  # Посещение корзины с последующим оформлением заказа
  g1 = ['Посещение корзины с добавленными товарами', 'Ввод промокода']

  # 1е шаги оформления
  g2 = ['Клик на кнопку Перейти к оформлению', 'Выбор радиобаттона eSIM', 'Выбор радиобаттона обычной SIM']

  # Изменение тарифа
  g3 = ['Выбор другого тарифа или номера в корзине']

  # Ввод личных данных
  g4 = ['Ввод контактного номера', 'Ввод имени', 'Ввод email',
        'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)', 'Клик на кнопку Продолжить 2 (Экран выбора региона)']

  # Оформление доставки
  g5 = ['Клик на кнопку Продолжить 3 (Экран выбора способа получения)', 'Клик на кнопку Продолжить 4 (Экран выбора адреса доставки/самовывоза)']

  # Заполнение паспортных данных
  g6 = ['Включение блока паспортных данных', 'Успешное заполнение всех полей паспортных данных']

  # Покупка
  g7 = ['Покупка']

  if event in g1:
    return 1
  elif event in g2:
    return 2
  elif event in g3:
    return 3
  elif event in g4:
    return 4
  elif event in g5:
    return 5
  elif event in g6:
    return 6
  elif event in g7:
    return 7

In [223]:
def good_events(arr):
    ''' Функция убирает те действия, которые повторяются, а также циклы повторяющихся действий. '''
    ans = [arr[0]]
    m = arr[0]
    for i in range(1, len(arr)):
        if arr[i] > arr[i-1] and arr[i] > m:
            ans.append(arr[i])
            m = arr[i]
    return ans

In [224]:
def add_event(line: str):
    ans = ''
    minimum = 1
    maximum = max(line)
    for i in range(int(minimum), int(maximum)+1):
        i = str(i)
        ans = ans + i if ((i in '36') and (i in line)) or (i not in '36') else ans
    return ans


In [225]:
def sessia_name(df_sessions):
  df_sessions = df_sessions[['client_id', 'event_name', 'event_time']]
  df_sessions = df_sessions.sort_values('event_time')
  df_sessions['event_name'] = df_sessions.event_name.apply(lambda x: make_group(x))
  df_sessions = df_sessions.groupby(['client_id'])['event_name'].agg(lambda x: good_events(x.to_list())).reset_index().sort_values('event_name')

  return df_sessions

In [226]:
def seesia_right_name(df_sessions):
  df_sessions_buy = df_sessions.copy()

  # Сессии без покупки
  df_sessions = df_sessions[df_sessions.event_name.apply(lambda x: 7 not in x)]

  # Сессии с покупкой
  df_sessions_buy = df_sessions_buy[df_sessions_buy.event_name.apply(lambda x: 7 in x)]
  df_sessions_buy_copy = df_sessions_buy.copy()

  df_sessions_buy_copy.event_name = df_sessions_buy_copy.event_name.apply(lambda x: x[0:(x.index(7)+1)])
  df_sessions_buy_copy = df_sessions_buy_copy[df_sessions_buy_copy['event_name'].apply(lambda x: len(x) > 0)]
  df_sessions_buy.event_name = df_sessions_buy.event_name.apply(lambda x: x[(x.index(7)+1):])
  df_sessions_buy = df_sessions_buy[df_sessions_buy['event_name'].apply(lambda x: len(x) > 0)]

  # Объединение всех сессий в 1 датафрейм
  df_sessions_buy = pd.concat([df_sessions_buy, df_sessions_buy_copy]).reset_index()
  df_sessions = pd.concat([df_sessions_buy, df_sessions]).reset_index()[['client_id', 'event_name']]

  # Заполение обязательных действий в случаи их пропусков
  df_sessions.event_name = df_sessions.event_name.apply(lambda x: ''.join(str(elem) for elem in x))
  df_sessions['event_name'] = df_sessions.event_name.apply(lambda x: add_event(x))

  return df_sessions

### подтверждение

In [227]:
data = pd.read_csv('Tele2_Clean.csv')
data.head()

,client_id,session_id,event_time,event_name,region_name,age,gender
0,117251283.16423236,5988994287612526860,2022-11-01 12:23:11,Покупка,high_td,25 - 39 лет,Male
1,117251283.16423236,5988994287612526860,2022-11-01 12:22:49,Клик на кнопку Продолжить 3 (Экран выбора спос...,high_td,25 - 39 лет,Male
2,117251283.16423236,5988994287612526860,2022-11-01 12:22:19,Ввод контактного номера,high_td,25 - 39 лет,Male
3,117251283.16423236,5988994287612526860,2022-11-01 12:22:59,Клик на кнопку Продолжить 4 (Экран выбора адре...,high_td,25 - 39 лет,Male
4,117251283.16423236,5988994287612526860,2022-11-01 12:22:21,Ввод имени,high_td,25 - 39 лет,Male


In [228]:
gender = ['Male', 'Female']
age = ['25 - 39 лет', '18 - 24 года', '54+', '40 - 54 года', 'До 18 лет']
region = list(data.region_name.unique())

In [236]:
general = data[(data['age'] != 'До 18 лет')]

In [256]:
ans = []
for sex in gender:
    for reg in region:
        viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == sex) & (data['region_name'] == reg)
        ]
        # формируем список выборочных конверсий
        viborka = sessia_name(viborka)
        viborka = seesia_right_name(viborka)
        event_count = []

        for i in range(1, 8):
            viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
            if i != 6 and i != 3:
                event_count.append(viborka['d_'+str(i)].sum())

        gsov = general
        gsov = sessia_name(gsov)
        gsov = seesia_right_name(gsov)
        event_count_g = []

        for i in range(1, 8):
            gsov['d_'+str(i)] = gsov['event_name'].apply(lambda x: x.count(str(i)))
            if i != 6 and i != 3: 
                event_count_g.append(gsov['d_'+str(i)].sum())
        # 1 2 4 5 7 event_count
        convers_v = {}
        convers_g = {}
            
        for i in range(1, 5):
            convers_v[f"{i}->{i + 1}"] = event_count[i] / event_count[i - 1]
            convers_g[f"{i}->{i + 1}"] = event_count_g[i] / event_count_g[i - 1]
        for i in range(4):
            if convers_v[f"{i + 1}->{i + 2}"] != 'not enough':
                z = (convers_v[f"{i + 1}->{i + 2}"] - convers_g[f"{i + 1}->{i + 2}"]) \
                / ((convers_g[f"{i + 1}->{i + 2}"] * (1 - convers_g[f"{i + 1}->{i + 2}"]) / event_count[i]) ** (1 / 2))
                # if z < -1.96 or z > 1.96:
                ans.append([sex, reg, z, f"{i + 1}->{i + 2}", event_count[i], convers_v[f"{i + 1}->{i + 2}"],
                            convers_g[f"{i + 1}->{i + 2}"]])
ans.sort(key=lambda x: abs(x[2]), reverse=True)

In [257]:
print(*ans, sep='\n')

['Male', 'low_td', -2.8891834307049136, '1->2', 335, 0.41492537313432837, 0.493845871222559]
['Male', 'Spb', -2.347825147380819, '2->3', 88, 0.375, 0.5001394959546173]
['Male', 'Msk', -1.8223578488888093, '3->4', 171, 0.7953216374269005, 0.845667534399405]
['Female', 'high_td', 1.7712411489977065, '4->5', 379, 0.8601583113456465, 0.8256376429199648]
['Female', 'Spb', -1.588984736745338, '1->2', 201, 0.43781094527363185, 0.493845871222559]
['Female', 'Spb', -1.4892790679978027, '4->5', 33, 0.7272727272727273, 0.8256376429199648]
['Male', 'Spb', -1.4007604254754034, '3->4', 33, 0.7575757575757576, 0.845667534399405]
['Male', 'low_td', -1.2995313648210312, '3->4', 66, 0.7878787878787878, 0.845667534399405]
['Female', 'low_td', 1.1426389710015352, '3->4', 75, 0.8933333333333333, 0.845667534399405]
['Female', 'Msk', -1.1194957136850825, '3->4', 167, 0.8143712574850299, 0.845667534399405]
['Male', 'high_td', 1.077648642898053, '3->4', 428, 0.8644859813084113, 0.845667534399405]
['Female', 'S

In [258]:
def norm_GtG(x):
    if x == '2->3':
        return '2->4'
    if x == '3->4':
        return '4->5'
    if x == '4->5':
        return '5->7'
    return x
    
a = pd.DataFrame(ans)
a.columns = ['sex', 'telephone_density', 'z-score', 'GtG', 'N', 'corr_v', 'corr_g']
a['GtG'] = a['GtG'].apply(lambda x: norm_GtG(x))
a.to_csv('sex_reg_less100.csv')

In [ ]:
a

,sex,telephone_density,z-score,GtG,N,corr_v,corr_g
0,Male,low_td,-2.461429,1->2,335,0.414925,0.482124
1,Male,high_td,-2.442845,2->4,876,0.488584,0.529779
2,Female,Msk,-1.497557,1->2,694,0.492795,0.521193
3,Female,Msk,1.469019,2->4,342,0.488304,0.448795
4,Female,high_td,-1.112676,4->5,444,0.853604,0.871287
5,Male,high_td,-1.072189,1->2,1813,0.483177,0.495767
6,Female,Msk,1.027216,5->7,136,0.794118,0.756303
7,Male,Msk,1.023307,4->5,171,0.795322,0.761996
8,Female,high_td,0.732916,5->7,379,0.860158,0.846591
9,Female,high_td,0.693505,1->2,1725,0.501449,0.493101


In [240]:
ans_2 = []
for reg in region:
    # подсчет действий выборки стариков данного региона
    viborka = data[((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & (data['region_name'] == reg)]
    viborka = sessia_name(viborka)
    viborka = seesia_right_name(viborka)

    event_count = []
    for i in range(1, 8):
        viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
        if i != 6 and i != 3:
            event_count.append(viborka['d_'+str(i)].sum())
    
    # подсчет действий молодых данного региона
    gsov = general
    gsov = sessia_name(gsov)
    gsov = seesia_right_name(gsov)

    event_count_g = []
    for i in range(1, 8):
        gsov['d_'+str(i)] = gsov['event_name'].apply(lambda x: x.count(str(i)))
        if i != 6 and i != 3: 
            event_count_g.append(gsov['d_'+str(i)].sum())

    convers_v = {}
    convers_g = {}
          
    for i in range(1, 5):
        if event_count[i] >= 100 and event_count[i - 1] >= 100:
            convers_v[f"{i}->{i + 1}"] = event_count[i] / event_count[i - 1]
            convers_g[f"{i}->{i + 1}"] = event_count_g[i] / event_count_g[i - 1]
        else:
            convers_v[f"{i}->{i + 1}"] = 'not enough'
            convers_g[f"{i}->{i + 1}"] = 'not enough'

    for i in range(4):
        if convers_v[f"{i + 1}->{i + 2}"] != 'not enough':
            z = (convers_v[f"{i + 1}->{i + 2}"] - convers_g[f"{i + 1}->{i + 2}"]) \
            / ((convers_g[f"{i + 1}->{i + 2}"] * (1 - convers_g[f"{i + 1}->{i + 2}"]) / event_count[i]) ** (1 / 2))
            ans_2.append([reg, z, f"{i + 1}->{i + 2}", event_count[i], convers_v[f"{i + 1}->{i + 2}"],
                        convers_g[f"{i + 1}->{i + 2}"]])
    # подсчет действий 
    

In [241]:
ans_2.sort(key=lambda x: x[1])
print(*ans_2, sep='\n')

['low_td', -2.1341333401332028, '1->2', 661, 0.4523449319213313, 0.493845871222559]
['Msk', -2.109468370113924, '3->4', 337, 0.8041543026706232, 0.845667534399405]
['Msk', -1.4005323753646743, '4->5', 271, 0.7933579335793358, 0.8256376429199648]
['low_td', -0.9879596438838038, '2->3', 299, 0.47157190635451507, 0.5001394959546173]
['Spb', -0.8996306918753358, '1->2', 374, 0.47058823529411764, 0.493845871222559]
['high_td', -0.1610103748664557, '1->2', 3529, 0.49249079059223577, 0.493845871222559]
['low_td', -0.05574194751751064, '3->4', 141, 0.8439716312056738, 0.845667534399405]
['Msk', -0.04573844215145877, '2->3', 675, 0.49925925925925924, 0.5001394959546173]
['high_td', -0.011630985434895774, '2->3', 1738, 0.5, 0.5001394959546173]
['Msk', 0.47932968175984014, '1->2', 1349, 0.5003706449221645, 0.493845871222559]
['low_td', 0.9058051996745601, '4->5', 119, 0.8571428571428571, 0.8256376429199648]
['high_td', 1.1375799294034235, '3->4', 869, 0.859608745684695, 0.845667534399405]
['high_

In [242]:
def norm_GtG(x):
    if x == '2->3':
        return '2->4'
    elif x == '3->4':
        return '4->5'
    elif x == '4->5':
        return '5->7'
    else:
        return x
    

a = pd.DataFrame(ans_2)
a.columns = ['telephone_density', 'z-score', 'GtG', 'N', 'corr_v', 'corr_g']
a['GtG'] = a['GtG'].apply(lambda x: norm_GtG(x))
a.to_csv('only_reg.csv')

In [ ]:
a[a['GtG'] == '5->7'].head()

,telephone_density,z-score,GtG,N,corr_v,corr_g
7,high_td,-0.038900,5->7,747,0.842035,0.842553
9,Msk,1.094239,5->7,271,0.793358,0.765182
10,low_td,1.138867,5->7,119,0.857143,0.816754


### сравниваем только по возрасту

In [ ]:
ans = []
# подсчет действий выборки стариков данного региона
viborka = data[((data['age'] == '40 - 54 года') | (data['age'] == '54+'))]
viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)
event_count = []
for i in range(1, 8):
    viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
    if i != 6 and i != 3:
        event_count.append(viborka['d_'+str(i)].sum())
 
# подсчет действий молодых данного региона
gsov = general_18_39
gsov = sessia_name(gsov)
gsov = seesia_right_name(gsov)

event_count_g = []
for i in range(1, 8):
    gsov['d_'+str(i)] = gsov['event_name'].apply(lambda x: x.count(str(i)))
    if i != 6 and i != 3: 
        event_count_g.append(gsov['d_'+str(i)].sum())
convers_v = {}
convers_g = {}
          
for i in range(1, 5):
    if event_count[i] >= 100 and event_count[i - 1] >= 100:
        convers_v[f"{i}->{i + 1}"] = event_count[i] / event_count[i - 1]
        convers_g[f"{i}->{i + 1}"] = event_count_g[i] / event_count_g[i - 1]
    else:
        convers_v[f"{i}->{i + 1}"] = 'not enough'
        convers_g[f"{i}->{i + 1}"] = 'not enough'

for i in range(4):
    if convers_v[f"{i + 1}->{i + 2}"] != 'not enough':
        z = (convers_v[f"{i + 1}->{i + 2}"] - convers_g[f"{i + 1}->{i + 2}"]) \
        / ((convers_g[f"{i + 1}->{i + 2}"] * (1 - convers_g[f"{i + 1}->{i + 2}"]) / event_count[i]) ** (1 / 2))
        ans.append([reg, z, f"{i + 1}->{i + 2}", event_count[i], convers_v[f"{i + 1}->{i + 2}"],
                    convers_g[f"{i + 1}->{i + 2}"]])
# подсчет действий 

In [ ]:
print(*ans, sep='\n')

['Spb', -1.5076419667236867, '1->2', 5912, 0.48832882273342354, 0.49813270260176773]
['Spb', -1.8423703841596777, '2->3', 2887, 0.4911673016972636, 0.5083093839810071]
['Spb', -0.3745072602408239, '3->4', 1418, 0.842031029619182, 0.8456243854473943]
['Spb', 0.6372369272248263, '4->5', 1194, 0.8291457286432161, 0.8220930232558139]


### сравниваем только по полу

In [ ]:
ans = []
# подсчет действий выборки стариков данного региона
viborka = data[(data['gender'] == 'Male')]
viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)
event_count = []
for i in range(1, 8):
    viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
    if i != 6 and i != 3:
        event_count.append(viborka['d_'+str(i)].sum())
 
# подсчет действий молодых данного региона
gsov = data[(data['gender'] == 'Female')]
gsov = sessia_name(gsov)
gsov = seesia_right_name(gsov)

event_count_g = []
for i in range(1, 8):
    gsov['d_'+str(i)] = gsov['event_name'].apply(lambda x: x.count(str(i)))
    if i != 6 and i != 3: 
        event_count_g.append(gsov['d_'+str(i)].sum())
convers_v = {}
convers_g = {}
          
for i in range(1, 5):
    if event_count[i] >= 100 and event_count[i - 1] >= 100:
        convers_v[f"{i}->{i + 1}"] = event_count[i] / event_count[i - 1]
        convers_g[f"{i}->{i + 1}"] = event_count_g[i] / event_count_g[i - 1]
    else:
        convers_v[f"{i}->{i + 1}"] = 'not enough'
        convers_g[f"{i}->{i + 1}"] = 'not enough'

for i in range(4):
    if convers_v[f"{i + 1}->{i + 2}"] != 'not enough':
        z = (convers_v[f"{i + 1}->{i + 2}"] - convers_g[f"{i + 1}->{i + 2}"]) \
        / ((convers_g[f"{i + 1}->{i + 2}"] * (1 - convers_g[f"{i + 1}->{i + 2}"]) / event_count[i]) ** (1 / 2))
        ans.append([reg, z, f"{i + 1}->{i + 2}", event_count[i], convers_v[f"{i + 1}->{i + 2}"],
                    convers_g[f"{i + 1}->{i + 2}"]])
# подсчет действий 
print(*ans, sep='\n')

['Spb', -0.4271533456803065, '1->2', 11534, 0.49488468874631525, 0.4968733249955333]
['Spb', 1.6461381345909978, '2->3', 5708, 0.509635599159075, 0.4987414599065085]
['Spb', -1.067053679623494, '3->4', 2909, 0.8418700584393263, 0.8489545782263879]
['Spb', -0.9014657114117758, '4->5', 2449, 0.8211514904042466, 0.8280254777070064]


In [ ]:
data.region_name.unique()

array(['high_td', 'Msk', 'low_td', 'Spb'], dtype=object)

### сравниваем только конверсии регионов

In [ ]:
msk = data[data['region_name'] == 'Msk']
spb = data[data['region_name'] == 'Spb']
high = data[data['region_name'] == 'high_td']
low = data[data['region_name'] == 'low_td']

msk = sessia_name(msk)
msk = seesia_right_name(msk)

spb = sessia_name(spb)
spb = seesia_right_name(spb)

high = sessia_name(high)
high = seesia_right_name(high)

low = sessia_name(low)
low = seesia_right_name(low)

event_count_msk = []
event_count_spb = []
event_count_high = []
event_count_low = []

for i in range(1, 8):

    msk['d_'+str(i)] = msk['event_name'].apply(lambda x: x.count(str(i)))
    spb['d_'+str(i)] = spb['event_name'].apply(lambda x: x.count(str(i)))
    high['d_'+str(i)] = high['event_name'].apply(lambda x: x.count(str(i)))
    low['d_'+str(i)] = low['event_name'].apply(lambda x: x.count(str(i)))

    if i != 6 and i != 3:
        event_count_msk.append(msk['d_'+str(i)].sum())
        event_count_spb.append(spb['d_'+str(i)].sum())
        event_count_high.append(high['d_'+str(i)].sum())
        event_count_low.append(low['d_'+str(i)].sum())
    
convers_msk = {}
convers_spb = {}
convers_high = {}
convers_low = {}
          
for i in range(1, 5):
    convers_msk[f"{i}->{i + 1}"] = event_count_msk[i] / event_count_msk[i - 1]
    convers_spb[f"{i}->{i + 1}"] = event_count_spb[i] / event_count_spb[i - 1]
    convers_high[f"{i}->{i + 1}"] = event_count_high[i] / event_count_high[i - 1]
    convers_low[f"{i}->{i + 1}"] = event_count_low[i] / event_count_low[i - 1]

print('msk', convers_msk)
print('spb', convers_spb)
print('high', convers_high)
print('low', convers_low)  


msk {'1->2': 0.5151232262882748, '2->3': 0.47517216382747374, '3->4': 0.7871853546910755, '4->5': 0.7742248062015504}
spb {'1->2': 0.4966292134831461, '2->3': 0.48114630467571645, '3->4': 0.8338557993730408, '4->5': 0.8157894736842105}
high {'1->2': 0.49093644227153066, '2->3': 0.509575609008733, '3->4': 0.8698135898977751, '4->5': 0.8444521258209471}
low {'1->2': 0.4661084529505582, '2->3': 0.504704875962361, '3->4': 0.8491525423728814, '4->5': 0.8243512974051896}


### сравниваем пожилых мужчин и женщин со всеми людьми 18-39

In [ ]:
ans = []
for sex in gender:
    for reg in region:
        viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == sex) & (data['region_name'] == reg)
        ]
        # формируем список выборочных конверсий
        viborka = sessia_name(viborka)
        viborka = seesia_right_name(viborka)
        event_count = []

        for i in range(1, 8):
            viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
            if i != 6 and i != 3:
                event_count.append(viborka['d_'+str(i)].sum())

        if True:
            # print(sex, reg, *event_count, '|    40+')
            # получим выборочные конверсии для группы другого возраста из того же региона
            gsov = general
            gsov = sessia_name(gsov)
            gsov = seesia_right_name(gsov)
            event_count_g = []

            for i in range(1, 8):
                gsov['d_'+str(i)] = gsov['event_name'].apply(lambda x: x.count(str(i)))
                if i != 6 and i != 3: 
                    event_count_g.append(gsov['d_'+str(i)].sum())
            # 1 2 4 5 7 event_count
            convers_v = {}
            convers_g = {}
            
            for i in range(1, 5):
                if event_count[i] >= 100 and event_count[i - 1] >= 100:
                    convers_v[f"{i}->{i + 1}"] = event_count[i] / event_count[i - 1]
                    convers_g[f"{i}->{i + 1}"] = event_count_g[i] / event_count_g[i - 1]
                else:
                    convers_v[f"{i}->{i + 1}"] = 'not enough'
                    convers_g[f"{i}->{i + 1}"] = 'not enough'
            
            for i in range(4):
                if convers_v[f"{i + 1}->{i + 2}"] != 'not enough':
                    z = (convers_v[f"{i + 1}->{i + 2}"] - convers_g[f"{i + 1}->{i + 2}"]) \
                    / ((convers_g[f"{i + 1}->{i + 2}"] * (1 - convers_g[f"{i + 1}->{i + 2}"]) / event_count[i]) ** (1 / 2))
                    # if z < -1.96 or z > 1.96:
                    ans.append([sex, reg, z, f"{i + 1}->{i + 2}", event_count[i], convers_v[f"{i + 1}->{i + 2}"],
                                convers_g[f"{i + 1}->{i + 2}"]])
ans.sort(key=lambda x: abs(x[2]), reverse=True)
print(*ans, sep='\n')

['Male', 'low_td', -3.0459096118174336, '1->2', 335, 0.41492537313432837, 0.49813270260176773]
['Female', 'high_td', 1.937723271011359, '4->5', 379, 0.8601583113456465, 0.8220930232558139]
['Male', 'Msk', -1.8205879643414689, '3->4', 171, 0.7953216374269005, 0.8456243854473943]
['Male', 'high_td', -1.2736120551525543, '1->2', 1813, 0.48317705460562604, 0.49813270260176773]
['Male', 'high_td', -1.1677692628421683, '2->3', 876, 0.4885844748858447, 0.5083093839810071]
['Female', 'Msk', -1.1178245070190511, '3->4', 167, 0.8143712574850299, 0.8456243854473943]
['Male', 'high_td', 1.0799961835259853, '3->4', 428, 0.8644859813084113, 0.8456243854473943]
['Male', 'Msk', -0.8530773643817214, '4->5', 136, 0.7941176470588235, 0.8220930232558139]
['Female', 'Msk', -0.8530773643817214, '4->5', 136, 0.7941176470588235, 0.8220930232558139]
['Female', 'Msk', -0.7400275528977853, '2->3', 342, 0.48830409356725146, 0.5083093839810071]
['Male', 'Msk', 0.48564197277186777, '1->2', 658, 0.5075987841945289, 

### сравним 1->7 у Male low_td и ген совокупности

In [243]:
viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == 'Male') & (data['region_name'] == 'low_td')
        ]

gen_1_7 = len(general[general['event_name'] == 'Покупка']) / \
    len(general[general['event_name'] == 'Посещение корзины с добавленными товарами'])
vib_1_7 = len(viborka[viborka['event_name'] == 'Покупка']) / \
    len(viborka[viborka['event_name'] == 'Посещение корзины с добавленными товарами'])
z = (vib_1_7 - gen_1_7) \
    / ((gen_1_7 * (1 - gen_1_7) / len(viborka[viborka['event_name'] == 'Посещение корзины с добавленными товарами'])) ** (1 / 2))
z

-0.48196945337559155

In [244]:
print('Значение конверсии 1->7 в ген.совокупности', gen_1_7, "\nЗначение 40+ из low=", vib_1_7, '\nZ-score=', z)

Значение конверсии 1->7 в ген.совокупности 0.1891964225974084 
Значение 40+ из low= 0.18207681365576103 
Z-score= -0.48196945337559155


### сравним 1->7 у Male high_td и ген совокупности

In [245]:
viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == 'Male') & (data['region_name'] == 'high_td')
        ]

gen_1_7 = len(general[general['event_name'] == 'Покупка']) / \
    len(general[general['event_name'] == 'Посещение корзины с добавленными товарами'])
vib_1_7 = len(viborka[viborka['event_name'] == 'Покупка']) / \
    len(viborka[viborka['event_name'] == 'Посещение корзины с добавленными товарами'])
z = (vib_1_7 - gen_1_7) \
    / ((gen_1_7 * (1 - gen_1_7) / len(viborka[viborka['event_name'] == 'Посещение корзины с добавленными товарами'])) ** (1 / 2))

In [247]:
print('Значение конверсии в 1->7 ген.совокупности=', gen_1_7, "\nЗначение 40+ из high_td=", vib_1_7, '\nZ-score=', z)

Значение конверсии в 1->7 ген.совокупности= 0.1891964225974084 
Значение 40+ из high_td= 0.1967509025270758 
Z-score= 1.2011415374038799


### сравним 2->7 у Male high_td и ген совокупности

In [263]:
viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == 'Male') & (data['region_name'] == 'high_td')
        ]

gen_2_7 = len(general[general['event_name'] == 'Покупка']) / \
    len(general[general['event_name'] == 'Клик на кнопку Перейти к оформлению'])
vib_2_7 = len(viborka[viborka['event_name'] == 'Покупка']) / \
    len(viborka[viborka['event_name'] == 'Клик на кнопку Перейти к оформлению'])
z = (vib_2_7 - gen_2_7) \
    / ((gen_2_7 * (1 - gen_2_7) / len(viborka[viborka['event_name'] == 'Клик на кнопку Перейти к оформлению'])) ** (1 / 2))

In [264]:
print('Значение конверсии 2->7 в ген.совокупности=', gen_2_7, "\nЗначение 40+ 2-> 7 из high_td=", vib_2_7, '\nZ-score=', z)

Значение конверсии 2->7 в ген.совокупности= 0.6382514483485066 
Значение 40+ 2-> 7 из high_td= 0.68125 
Z-score= 2.9947704109258164


### сравним 2->7 у Male low_td и ген совокупности

In [265]:
viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == 'Male') & (data['region_name'] == 'low_td')
        ]

gen_2_7 = len(general[general['event_name'] == 'Покупка']) / \
    len(general[general['event_name'] == 'Клик на кнопку Перейти к оформлению'])
vib_2_7 = len(viborka[viborka['event_name'] == 'Покупка']) / \
    len(viborka[viborka['event_name'] == 'Клик на кнопку Перейти к оформлению'])
z = (vib_2_7 - gen_2_7) \
    / ((gen_2_7 * (1 - gen_2_7) / len(viborka[viborka['event_name'] == 'Клик на кнопку Перейти к оформлению'])) ** (1 / 2))

In [266]:
print('Значение конверсии 2->7 в ген.совокупности=', gen_2_7, "\nЗначение 40+ 2-> 7 из low_td=", vib_2_7, '\nZ-score=', z)

Значение конверсии 2->7 в ген.совокупности= 0.6382514483485066 
Значение 40+ 2-> 7 из low_td= 0.6464646464646465 
Z-score= 0.24051689522317704


### сравним 4->7 у Male low_td и ген совокупности

In [267]:
viborka = data[
            ((data['age'] == '40 - 54 года') | (data['age'] == '54+')) & 
            (data['gender'] == 'Male') & (data['region_name'] == 'low_td')
        ]

gen_4_7 = len(general[general['event_name'] == 'Покупка']) / \
    len(general[general['event_name'] == 'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)'])
vib_4_7 = len(viborka[viborka['event_name'] == 'Покупка']) / \
    len(viborka[viborka['event_name'] == 'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)'])
z = (vib_4_7 - gen_4_7) \
    / ((gen_4_7 * (1 - gen_4_7) / len(viborka[viborka['event_name'] == 'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)'])) ** (1 / 2))

In [268]:
print('Значение конверсии 4-> 7 в ген.совокупности=', gen_1_7, "\nЗначение 40+ 4-> 7 из low_td=", vib_1_7, '\nZ-score=', z)

Значение конверсии 4-> 7 в ген.совокупности= 0.1891964225974084 
Значение 40+ 4-> 7 из low_td= 0.1967509025270758 
Z-score= 1.039479294424401
